In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import os
import re
import string
import nltk
nltk.download('punkt')
from nltk import word_tokenize,sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
story_path = "/content/drive/MyDrive/sherlock/"

def read_all_stories(story_path):
  txt = []
  for _, _, files in os.walk(story_path):
    for file in files:
      with open(story_path+file) as f:
        for line in f:
          line = line.strip()
          if line == '----------':
            break
          if line!='':
            txt.append(line)

  return txt

stories = read_all_stories(story_path)
print("number of lines in the dataset = ", len(stories))

number of lines in the dataset =  215021


In [ ]:
def clean_txt(txt):
  cleaned_txt = []
  for line in txt:
    line = line.lower()
    line = re.sub(r"[,\"\'!@#$%^&*(){}?/;`~:<>+=-\\]", "", line)
    tokens = word_tokenize(line)
    words = [word for word in tokens if word.isalpha()]
    cleaned_txt+=words

  return cleaned_txt

In [ ]:
cleaned_stories = clean_txt(stories)
print("Number of total words = ", len(cleaned_stories))

Number of total words =  2323140


In [ ]:
cleaned_stories

['the',
 'adventure',
 'of',
 'the',
 'abbey',
 'grange',
 'arthur',
 'conan',
 'doyle',
 'it',
 'was',
 'on',
 'a',
 'bitterly',
 'cold',
 'and',
 'frosty',
 'morning',
 'during',
 'the',
 'winter',
 'of',
 'that',
 'i',
 'was',
 'awakened',
 'by',
 'a',
 'tugging',
 'at',
 'my',
 'shoulder',
 'it',
 'was',
 'holmes',
 'the',
 'candle',
 'in',
 'his',
 'hand',
 'shone',
 'upon',
 'his',
 'eager',
 'stooping',
 'face',
 'and',
 'told',
 'me',
 'at',
 'a',
 'glance',
 'that',
 'something',
 'was',
 'amiss',
 'come',
 'watson',
 'come',
 'he',
 'cried',
 'the',
 'game',
 'is',
 'afoot',
 'not',
 'a',
 'word',
 'into',
 'your',
 'clothes',
 'and',
 'come',
 'ten',
 'minutes',
 'later',
 'we',
 'were',
 'both',
 'in',
 'a',
 'cab',
 'and',
 'rattling',
 'through',
 'the',
 'silent',
 'streets',
 'on',
 'our',
 'way',
 'to',
 'charing',
 'cross',
 'station',
 'the',
 'first',
 'faint',
 'winters',
 'dawn',
 'was',
 'beginning',
 'to',
 'appear',
 'and',
 'we',
 'could',
 'dimly',
 'see',
 '

In [ ]:
n_gram = 2
markov_model = {}
for i in range(100):
    curr_state, next_state = "", ""
    # print('printing', curr_state, next_state)
    for j in range(n_gram):
        curr_state += cleaned_stories[i+j] + " "
        next_state += cleaned_stories[i+j+n_gram] + " "
        # print(curr_state, next_state)
    curr_state = curr_state[:-1]
    next_state = next_state[:-1]
    # print(curr_state, next_state)
    print(markov_model)
    if curr_state not in markov_model:
        markov_model[curr_state] = {}
        markov_model[curr_state][next_state] = 1
    else:
        if next_state in markov_model[curr_state]:
            markov_model[curr_state][next_state] += 1
        else:
            markov_model[curr_state][next_state] = 1

for curr_state, transition in markov_model.items():
    total = sum(transition.values())
    # print(total)
    for state, count in transition.items():
        markov_model[curr_state][state] = count/total

{}
{'the adventure': {'of the': 1}}
{'the adventure': {'of the': 1}, 'adventure of': {'the abbey': 1}}
{'the adventure': {'of the': 1}, 'adventure of': {'the abbey': 1}, 'of the': {'abbey grange': 1}}
{'the adventure': {'of the': 1}, 'adventure of': {'the abbey': 1}, 'of the': {'abbey grange': 1}, 'the abbey': {'grange arthur': 1}}
{'the adventure': {'of the': 1}, 'adventure of': {'the abbey': 1}, 'of the': {'abbey grange': 1}, 'the abbey': {'grange arthur': 1}, 'abbey grange': {'arthur conan': 1}}
{'the adventure': {'of the': 1}, 'adventure of': {'the abbey': 1}, 'of the': {'abbey grange': 1}, 'the abbey': {'grange arthur': 1}, 'abbey grange': {'arthur conan': 1}, 'grange arthur': {'conan doyle': 1}}
{'the adventure': {'of the': 1}, 'adventure of': {'the abbey': 1}, 'of the': {'abbey grange': 1}, 'the abbey': {'grange arthur': 1}, 'abbey grange': {'arthur conan': 1}, 'grange arthur': {'conan doyle': 1}, 'arthur conan': {'doyle it': 1}}
{'the adventure': {'of the': 1}, 'adventure of': 

In [ ]:
def make_markov_model(cleaned_stories, n_gram=2):
    markov_model = {}
    for i in range(len(cleaned_stories)-n_gram-1):
        curr_state, next_state = "", ""
        for j in range(n_gram):
            curr_state += cleaned_stories[i+j] + " "
            next_state += cleaned_stories[i+j+n_gram] + " "
        curr_state = curr_state[:-1]
        next_state = next_state[:-1]
        if curr_state not in markov_model:
            markov_model[curr_state] = {}
            markov_model[curr_state][next_state] = 1
        else:
            if next_state in markov_model[curr_state]:
                markov_model[curr_state][next_state] += 1
            else:
                markov_model[curr_state][next_state] = 1

    for curr_state, transition in markov_model.items():
        total = sum(transition.values())
        for state, count in transition.items():
            markov_model[curr_state][state] = count/total

    return markov_model

In [ ]:
markov_model = make_markov_model(cleaned_stories)

In [ ]:
markov_model

{'the adventure': {'of the': 0.8506944444444444,
  'of charles': 0.024305555555555556,
  'far from': 0.013888888888888888,
  'of black': 0.024305555555555556,
  'of wisteria': 0.024305555555555556,
  'of shoscombe': 0.017361111111111112,
  'was a': 0.013888888888888888,
  'of last': 0.010416666666666666,
  'as a': 0.010416666666666666,
  'i will': 0.010416666666666666},
 'adventure of': {'the abbey': 0.038461538461538464,
  'the three': 0.05944055944055944,
  'the blue': 0.024475524475524476,
  'the speckled': 0.024475524475524476,
  'the engineers': 0.024475524475524476,
  'the noble': 0.024475524475524476,
  'the beryl': 0.024475524475524476,
  'the copper': 0.024475524475524476,
  'the paradol': 0.013986013986013986,
  'the man': 0.013986013986013986,
  'charles augustus': 0.024475524475524476,
  'the empty': 0.038461538461538464,
  'the norwood': 0.024475524475524476,
  'the dancing': 0.024475524475524476,
  'the solitary': 0.024475524475524476,
  'the priory': 0.024475524475524476

In [ ]:
print("Number of States = ", len(markov_model.keys()))

Number of States =  208772


In [ ]:
print("All possible transitions from 'the game' state: \n")
print(markov_model['my god'])

All possible transitions from 'the game' state: 

{'my god': 0.039603960396039604, 'my sins': 0.039603960396039604, 'its watson': 0.039603960396039604, 'what an': 0.039603960396039604, 'helen it': 0.039603960396039604, 'i whispered': 0.039603960396039604, 'what shall': 0.039603960396039604, 'what a': 0.039603960396039604, 'he cried': 0.10891089108910891, 'cried our': 0.039603960396039604, 'it is': 0.07920792079207921, 'was there': 0.039603960396039604, 'its nancy': 0.039603960396039604, 'are you': 0.039603960396039604, 'gasped our': 0.039603960396039604, 'whats that': 0.0297029702970297, 'can there': 0.0297029702970297, 'i gasped': 0.0297029702970297, 'those screams': 0.0297029702970297, 'he whispered': 0.0297029702970297, 'to better': 0.0297029702970297, 'whoever would': 0.039603960396039604, 'shall i': 0.039603960396039604, 'how he': 0.039603960396039604}


In [ ]:
print(list(markov_model['my god'].keys()), list(markov_model['my god'].values()))

['my god', 'my sins', 'its watson', 'what an', 'helen it', 'i whispered', 'what shall', 'what a', 'he cried', 'cried our', 'it is', 'was there', 'its nancy', 'are you', 'gasped our', 'whats that', 'can there', 'i gasped', 'those screams', 'he whispered', 'to better', 'whoever would', 'shall i', 'how he'] [0.039603960396039604, 0.039603960396039604, 0.039603960396039604, 0.039603960396039604, 0.039603960396039604, 0.039603960396039604, 0.039603960396039604, 0.039603960396039604, 0.10891089108910891, 0.039603960396039604, 0.07920792079207921, 0.039603960396039604, 0.039603960396039604, 0.039603960396039604, 0.039603960396039604, 0.0297029702970297, 0.0297029702970297, 0.0297029702970297, 0.0297029702970297, 0.0297029702970297, 0.0297029702970297, 0.039603960396039604, 0.039603960396039604, 0.039603960396039604]


In [ ]:
def generate_story(markov_model, limit=100, start = 'my god'):
  n = 0
  curr_state = start
  next_state = None
  story = ""
  story += curr_state+" "
  while n<limit:
    next_state = random.choices(
        list(markov_model[curr_state].keys()),
        list(markov_model[curr_state].values())
    )
    # print(next_state)
    curr_state = next_state[0]
    story+= curr_state+" "
    n+=1

  return story

In [ ]:
generate_story(markov_model, limit = 5)

'my god whoever would have thought that it is more than a '

In [ ]:
for i in range(20):
  print(str(i), generate_story(markov_model, start = "dear holmes", limit = 8))

0 dear holmes i thought that were true the gentleman however seeing perhaps the look rather than in his 
1 dear holmes he has treated me she shot her arms out from under the shadow of a doubt 
2 dear holmes am i to go to bed and the public shall know them and it is also 
3 dear holmes and tell him but he took an orange from the face or the poise of the 
4 dear holmes i ejaculated no for you in holmes without flying away after theories and fancies you are 
5 dear holmes i exclaimed and this i joined it at last watson british government you would come the 
6 dear holmes i should be glad to have revenge upon them could speak english will be offered before 
7 dear holmes oh yes with the official police force i thank you for this sort of wild despairing 
8 dear holmes am i allowed to cooperate with him and this man stimulated my curiosity and i must 
9 dear holmes i thought that if she had been of the most singular and interesting study this is 
10 dear holmes i ejaculated surely said he wi

In [ ]:
for i in range(20):
  print(str(i), generate_story(markov_model, start = "my dear", limit = 8))

0 my dear watson and you will have the great kindness if you cast your mind back can you 
1 my dear son the son as far as we could and soon the whole matter out again my 
2 my dear fellow i cried approaching him stand back stand right back said he but even the bark 
3 my dear sir dont look so scared he muttered in a very careful examination of the freedom which 
4 my dear watson there is no reason why marie had left her mistress because she was fondly attached 
5 my dear watson that we can always bait our line but never in that armchair doctor and give 
6 my dear fellow it would be wise to secure our retreat through the outer door closed and some 
7 my dear chap i had seen him get him here he fainted twice upon the way half a 
8 my dear fellow i congrat to milvertons housemaid good heavens are you godfrey staunton what about him if 
9 my dear watson that we must prepare for the new wing and the old cook and the two 
10 my dear watson you improve all the time and the maid howells to tha

In [ ]:
for i in range(20):
    print(str(i)+". ", generate_story(markov_model, start="i would", limit=5))

0.  i would always carry the case he continued to be expostulating with 
1.  i would i could see that our hands if i remember rightly 
2.  i would offer you a glass of beer he answered oh yes 
3.  i would have brought me out of the business of the advertisement 
4.  i would swear to you that this is what he had said 
5.  i would go with this lady this was quite too good to 
6.  i would give my fortune to any man succeeded in our object 
7.  i would offer you a beggarly sum which i offer her attentions 
8.  i would not have me he cried and sobbed like a child 
9.  i would only return etc the good steiler assured me that it 
10.  i would give so much to do with me i confess lady 
11.  i would have nothing more to say to discourage him someone else 
12.  i would only return etc the good steiler assured me that it 
13.  i would not have anticipated in some ways an instructive quarter of 
14.  i would have gone in the passage and again knocked but without 
15.  i would only ask you to come 

In [ ]:
print(generate_story(markov_model, start="the case", limit=100))

the case i guess most men have a little chat with watson that will be sam merton the boxer not a bad business watson which does not surprise me after a thousand pretty italian exclamations pouring from her and i watson we resume our conversation but it was his custom to smoke he answered raising his voice and figure being the same life and death to him and i understood what it consisted he was led to believe that i fancy that this lady in a i was in possession owing to the intersection of one of the police you may look for your friend baron von herling the chief difficulty in being directed to charlington heath and there sure enough just imagine what i felt it was amusing to notice how the consequential jones was already beginning to give us a chance of ever getting out again in an incisive and positive tone opened the door stepping out into the garden to the settee and blew little wavering rings of smoke curled through the room when barker arrived the door was locked on the inside poc